In [ ]:
import sys
from PyQt5.QtWidgets import QMainWindow, QApplication,QButtonGroup
from PyQt5 import uic
from math import exp,floor
from scipy.stats import norm
import yfinance as yf
import numpy as np

qtCreatorFile = "OptionPrice.ui"
Ui_MainWindow, QtBaseClass = uic.loadUiType(qtCreatorFile)

def explicit_method(S,K,r,q,T,sigma,M,N,callput):
    dt = T/N
    Smax = 2*K
    ds = Smax/M
    fc = np.zeros((N+1,M+1))
    fp = np.zeros((N+1,M+1))
    fhat = np.zeros((N,M+1))
    j = np.arange(0,M+1)
    fc[N,:] = np.maximum(j*ds-K,0)
    fp[N,:] = np.maximum(K-j*ds,0)
    # A matrix
    A = np.zeros((M+1,M+1))
    A[0,0] = 1
    A[M,M] = 1
    for j in range(1,M):
        A[j,j-1] = 0.5 * dt * (sigma**2 * j**2 - (r-q)* j)
        A[j,j] = 1 - dt * (sigma**2 * j**2 + r)
        A[j,j+1] = 0.5 * dt * (sigma**2 * j**2 + (r-q)* j)
    
    if callput == "call":
        for i in range(N-1,-1,-1):
            result = np.matmul(A, fc[i+1,:].reshape(-1, 1)).flatten()    
            fhat[i,:] = result
            for j in range(0,M+1):
                if j == 0:
                    fc[i,j] = 0
                elif j == M:
                    fc[i,j] = Smax - K * exp(-r * (N-i) * dt)
                else:
                    fc[i,j] = fhat[i,j]
    elif callput == "put":
        for i in range(N-1,-1,-1):
            result = np.matmul(A, fp[i+1,:].reshape(-1, 1)).flatten()     
            fhat[i,:] = result
            for j in range(0,M+1):
                if j == 0:
                    fp[i,j] = K * exp(-r * (N-i) * dt)
                elif j == M:
                    fp[i,j] = 0
                else:
                    fp[i,j] = fhat[i,j]
    k = floor(S/ds)
    if callput == "call":
        V = fc[0,k] + (fc[0,k+1] - fc[0,k])/ds * (S - k * ds)
    elif callput == "put":
        V = fp[0,k] + (fp[0,k+1] - fp[0,k])/ds * (S - k * ds)
    return V

class Main(QMainWindow, Ui_MainWindow):
    def __init__(self):        
        super().__init__()
        self.setupUi(self) 
        self.radioButton_Call.clicked.connect(self.Calculate)
        self.radioButton_Put.clicked.connect(self.Calculate)
        self.pushButton_Calculate.clicked.connect(self.Calculate)
        self.pushButton_GetPrice.clicked.connect(self.GetPrice)
        
    def GetPrice(self):
        x = self.lineEdit_Ticker.text()
        ticker = yf.Ticker(x)
        hist = ticker.history(period="1d")
        latest_close = hist["Close"].iloc[-1]
        self.lineEdit_StockPrice.setText(str(latest_close))
    
    def Calculate(self):
        S = float(self.lineEdit_StockPrice.text())
        K = float(self.lineEdit_StrikePrice.text())
        T= float(self.lineEdit_Maturity.text())/365
        sigma = float(self.lineEdit_StockVolatility.text())
        r = float(self.lineEdit_InterestRate.text())
        q = float(self.lineEdit_Dividend.text())
        M = int(self.lineEdit_M.text())
        N = int(self.lineEdit_N.text())
        if self.radioButton_Call.isChecked():
            callput = "call"
        else:
            callput = "put"

        option_price = explicit_method(S,K,r,q,T,sigma,M,N, callput)
        
        self.lineEdit_OptionPrice.setText(str(round(option_price,4)))
        
 
if __name__ == '__main__':
    app = QApplication(sys.argv)
    main = Main()
    main.show()
    sys.exit(app.exec_())

ValueError: could not convert string to float: ''